In [1]:
crop_calibrated = Crop("Maize", planting_date='05/01',
            # growth stages
            CalendarType = 1,
            SwitchGDD = 1,  #Convert calendar to gdd mode if inputs are given in calendar days (0 = No; 1 = Yes)
            ##Crop Development
            Emergence = 15, #15
            Senescence = 42, #42
            Maturity = 131, #131
            MaxRooting = 91, #91
            HIstart = 83,
            Flowering = 67,
            YldForm = 60,
            
            # canopy cover profile
            #CCx = 0.94,
            CGC = 13.7,
            CDC = 1.31,
    
            # after running swam
            CCx = 0.99,
            WPy = 75,
            Wp = 33.3641,
            Kcb = 1.1


            
    )


# run model
model_c = AquaCropModel(sim_start,sim_end,wdf,custom,crop_calibrated,initWC, irr_mngt)
model_c.run_model(till_termination=True) # run model till the end
model_c_et = model_c._outputs.water_flux
model_c_irr = model_c._outputs.final_stats
#model_c_water_storage = model_c._outputs.water_storage
#model_c_crp_grwth = model_c._outputs.crop_growth

 # make year a column 
model_c_irr = model_c_irr.assign(Year =  model_c_irr['Harvest Date (YYYY/MM/DD)'].dt.year)
model_c_irr = model_c_irr.rename(columns={
                   'Yield (tonne/ha)': 'Calib Yield (t/ha)',
                   'Seasonal irrigation (mm)': 'Calib Irrigation (mm)'
                   })

model_c_irr = model_c_irr[['Year', 'Calib Yield (t/ha)', 'Calib Irrigation (mm)']]



NameError: name 'Crop' is not defined

In [ ]:
# compare models
model_comp = model_uc_irr.merge(model_c_irr,on='Year').merge(sher_irrig_corn,on='Year').merge(sher_wimas, on = 'Year')
model_comp = model_comp.assign(YieldUSDA = model_comp['Value']*0.0673)


In [ ]:
#model_comp = model_comp[model_comp['Year'] != 2009]
# comparison - time series plot
irrig_colors = ['#FFA500', '#8abbdb', '#69a6d0', '#4892c6', '#367bac', '#356384', '#344b5b']

plt.style.use('default')
plt.plot(model_comp['Year'], model_comp['Calib Yield (t/ha)'],  label='Calibrated AqC', color = 'goldenrod', linewidth=3)
plt.plot(model_comp['Year'], model_comp['Uncalib Yield (t/ha)'],  label='Uncalibrated AqC', color = 'grey', linewidth=3)
plt.plot(model_comp['Year'], model_comp['YieldUSDA'],  label='USDA-NASS', color = '#2e8b57', linestyle='dashed')
plt.ylabel('Yield (t/ha)', fontsize=15)
plt.legend(loc='lower right', fontsize=12, frameon=False)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
#plt.savefig('results/visual

In [ ]:
# 1:1 plot
plt.scatter(model_comp['YieldUSDA'], model_comp['Calib Yield (t/ha)'],  label='Calibrated AqC', color = 'goldenrod', edgecolors = 'black', s = 50)
plt.scatter(model_comp['YieldUSDA'], model_comp['Uncalib Yield (t/ha)'],  label='Uncalibrated AqC', color = 'grey', edgecolors = 'black', s=50)
plt.plot([10,15],[10,15], color = 'black')
plt.legend(loc='lower right', fontsize=12, frameon=False)
plt.ylabel('AquaCrop Yield (t/ha)', fontsize=15)
plt.xlabel('USDA-NASS (t/ha)', fontsize=15)
plt.legend(loc='lower right', fontsize=12, frameon=False)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
#plt.savefig('results/visuals/yieldmodelcal_40SMT_11.png', format='png', dpi=600)